In [1]:
%matplotlib inline
import matplotlib
import scanpy as sc
import pandas as pd
import anndata
import convert_adata_to_scp as conv
import os
import ipywidgets as widgets
import requests
import widget_helpers as uh
import MetadataAdder as ma

/Users/nyquist/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Alexandria metadata annotation for scanpy or Seurat files

For this notebook, you are expected to have the following inputs:


* ONE AnnData (scanpy) object saved as an h5ad file with the following attributes: 
    * containing few enough cells that you are willing to save it in dense format
    * with some dimensionality reduction already run (ex. UMAP or tSNE)
    * with some metadata/clustering already done
    * at least one metadata column with a value describing the sample of each cell such that all cells from that sample have an identical value in that column
    
    
---OR---

* ONE Seurat .RData object containing:
    * ScaleData containing gene expression
    * seurat_obs@meta.data containing cell-level metadata
    * dimensionality reduction saved in the Seurat object


---AND---
* (optional) a per sample csv, txt, excel, or tsv file with the following attributes:
    * one row per sample
    * the values in at least one column are sample names, and these sample names exactly match the values in the sample column in the scanpy/seurat metadata
    
* (optional) a per donor csv, txt, excel, or tsv file with the following attributes:
    * This is useful if you have multiple samples per donor and some metadata that is the same each sample from the same donor
    * there should be a column in this file with values as donor identifiers that maps to a column in the sample csv or the scanpy/seurat object
    
    
    
    
    
## Troubleshooting:

If you are not seeing graphical output try running ``jupyter labextension install @jupyter-widgets/jupyterlab-manager``

## USER INPUT: Paths to files

In [3]:

# path to anndata/scanpy object, leave as "" if you are not using it
anndata_path = ""#"../../epithelial_cell_clustering.h5ad" #TODO allow for multiple objects!

# path to sample file, leave as "" if you are not using it
sample_metadata_path = ""

# path to donor file, leave as "" if you are not using it
donor_metadata_path = ""

# TODO: NOT SUPPORTED YET
# paths to cell level metadata files, leave as [] if you are not using it - this is for if you happened to save your cell metadata files separately
# these should be: cells as rows and metadata/clusters as columns
cell_metadata_paths = []

# path to .Rds seurat object, leave as "" if you are not using it
seurat_paths = ["/Users/nyquist/Dropbox (MIT)/Shalek Lab (Team folder conflict)/Projects/Gates/NIH.Vaccine.Route.Comparison.2019/Objects/Week25.All.Seurat.Rdata",
              "/Users/nyquist/Dropbox (MIT)/Shalek Lab (Team folder conflict)/Projects/Gates/NIH.Vaccine.Route.Comparison.2019/Objects/Week13.All.Seurat.Rdata"] #TODO allow for multiple objects!

# desired output directory full path (leave as "" if you want to output to the directory containing this notebook)
output_dir = ""

output_file_names = {}

In [4]:
# no editing required for this cell, just run it
mapping_options = {}
if len(anndata_path) > 0:
    adata = sc.read_h5ad(anndata_path)
    print(adata)
    mapping_options["anndata"] = adata
    mapping_options["cell level dataframe"] = adata.obs
if len(sample_metadata_path) > 0:
    mapping_options["sample_metadata_path"] = sample_metadata_path
    
if len(donor_metadata_path) > 0:
    mapping_options["donor_metadata_path"] = donor_metadata_path

if len(seurat_paths)  > 0:
    %load_ext rpy2.ipython
    
    # TODO - use rpy2 to read out metadata from seurat, hopefully make a function that accepts as input the name of this feature so this works for several versions of seurat
    #seurat_metadata = conv.get_metadata_from_seurat(seurat_paths, "meta.data")
    # TODO: because reading in seurat files a bunch of times might be way too slow, it could make sense for us to have one function that will open the file, save the expression and cluster files, and return the dataframe
    #mapping_options["cell level dataframe"] = seurat_metadata
    
'''
if len(cell_metadata_paths) > 0:
    if "cell level dataframe" in mapping_options:# if you already have other cell level info here
        for p in cell_metadata_paths:
            if p.split(".")[-1] =="csv":
                some_cell_level_metadata = pd.read_csv(p, index_col=0)
            else: # assume if it is not csv, it is tab-deliminated
                some_cell_level_metadata = pd.read_csv(p, index_col=0, sep="\t")
                
            if len(set(some_cell_level_metadata.index).intersection(set(mapping_options["cell level dataframe"].index))) < len(some_cell_level_metadata.index):
                print("Some new cells are not in the existing cell level metadata!! This is probably not going to work!")
            # you are going to overwrite the metadata columns in your other file if they are the same as the ones in this file
            for col in some_cell_level_metadata.columns:
                mapping_options["cell level dataframe"][col] = ""
                mapping_options["cell level dataframe"].loc[some_cell_level_metadata.index, ]
    if len(cell_metadata_paths) > 1: # you're doing different stuff if you need to merge these or if there is just one
        mapping_options["cell level files"] = cell_metadata_paths
    elif : # if you already have other cell level info here
        # if 
'''

'\nif len(cell_metadata_paths) > 0:\n    if "cell level dataframe" in mapping_options:# if you already have other cell level info here\n        for p in cell_metadata_paths:\n            if p.split(".")[-1] =="csv":\n                some_cell_level_metadata = pd.read_csv(p, index_col=0)\n            else: # assume if it is not csv, it is tab-deliminated\n                some_cell_level_metadata = pd.read_csv(p, index_col=0, sep="\t")\n                \n            if len(set(some_cell_level_metadata.index).intersection(set(mapping_options["cell level dataframe"].index))) < len(some_cell_level_metadata.index):\n                print("Some new cells are not in the existing cell level metadata!! This is probably not going to work!")\n            # you are going to overwrite the metadata columns in your other file if they are the same as the ones in this file\n            for col in some_cell_level_metadata.columns:\n                mapping_options["cell level dataframe"][col] = ""\n      

# Converting from R:

Now we go into R and convert to text files. The 'scp_save_seruat.R' will take as input the path or paths to seurat objects saved as .Rdata files. Unfortunately, Seurat does not maintain backwards compatibility when it comes to the seurat object data structure. Since everyone is using different versions of seruat, this code might require quite a bit of editing from you. It currently makes the following assumptions:


- EXPRESSION FILES: The expression value of interest is stored in seurat.object@data

- DIMENSIONALITY REDUCTION: The tsne or umap values are stored in seurat.object@meta.data as columns named 'X_umap1', 'X_umap2', 'X_tsne1', 'X_tsne2' - this script will only automatically check for those two dimensionality reductions. If you have other ones, you will need to g

- METADATA: The metadata is saved in seurat.object@meta.data - if there are columns that mean the same thing but have different names in your different seurat objects (ex. in object 1 you called the cell type column "celltype" and in object 2 you called the celltype column "cell.type") It will be best that you merge those. You will need to write your own code to do this
        

### IMPORTANT: This series of code blocks will automatically write files with your expression data and dimensionality reduction (cluster) results but not your metadata. This is expected to be passed back to python for annotation into the Alexandria format. Make sure you end up getting this file!!

In [ ]:
%%R -i seurat_paths -i output_dir
# Only run this if you data is in seurat
# If your objects are huge, you are going to have to do this outside of the notebook
source("scp_save_seurat.R")
# in case you have cells in your various objects that happen to have the same name, we add a prefix to the cell names.
# this array is the prefix to your cell names which should correspond to the order in your 'seurat_paths' variable
# keep this an array even if you only have one cell name

proj_names <- c("Week25","Week13") #EDIT THIS
output_prefix<-paste(output_dir, "SCP")

If your seurat objects meet all of the assumptions described above, run the following block:

Specifically:

- expression stored in seurat.object@data

- tsne or umap stored in seurat.object@meta.data as 'X_umap1' etc

- metadata is in seurat.object@meta.data and you only have one object or you don't mind dealing with misaligned column names later

In [ ]:
%%R -o merged_metadata 
# Run the line below if your seurat objects meet all the requirements above and you don't want to merge your metadata manually (or you only have one seurat object)
merged_metadata <- load_multiple_seurat_files(seurat_paths, proj_name= proj_names, output_prefix=output_prefix)



If you ran the above cell, skip down to 'OPTIONAL: Fix misaligned metadata'

## Seurat object step 1: Save expression files

If you have more than one seurat object, you will need to do this for each one. You can uncomment the for loop or just copy and paste the code a few times. Beware that the notebook might crash if you load in too much data, so it might be good to go through this whole process once per object (as long as you are saving your metadata as different variable names for each object)


### Changes from user:

If your data is not saved in your seurat object as seurat.object@data, you will need to change the 'exp.data' variable below

In [32]:
%%R  
i <- 1  # i is the index in the array 'seurat_paths'
#for(i in 1:length(seurat_paths)){
obj_<-load(seurat_paths[[path]])
obj <- get(obj_)
proj_name <- proj_names[i]
#    dim_red_types <- c("tsne", "umap") # if your dimensionality reduction is not saved in the format described in the text above this cell, remove the values from this array and run dimensionality reduction manually
# USER INPUT: if your data is not saved in obj@data, run the following chunk (replacing obj@data with the correct thing)
exp.data <-obj@data

exp_filename <- paste(output_prefix, "_norm_expression.txt.gz",sep="")
exp_df <- add_gene_column(exp.data,proj_name) 
write.csv.gz(x=exp_df, file=exp_filename, quote = FALSE,sep = "\t",col.names = TRUE)

#}
#

## Seurat object step 2: Save cluster files

This assumes that your seurat object is saved as 'obj' from the cell above

In [ ]:
%%R

# USER INPUT: change this if your dimensionality reduction info is not saved in @meta.data
dim_red_dataframe <- obj@meta.data

# USER INPUT: change these if the column names of your dimensionality reduction are not the ones used below
X_name <- "X_umap1"
Y_name <- "X_umap2"

# USER INPUT: change this to reflect the name of your dimensionality reduction type
cluster_file_prefix <- paste(output_prefix, proj_name, "umap", sep="_")

save_cluster_file(dim_red_dataframe, X_name, Y_name, proj_name)

## Seurat object step 3: Get the metadata!

In [ ]:
%%R -o merged_metadata 

# if your metadata is not in obj@meta.data, change this
seurat_metadata <- obj@meta.data


merged_metadata <- data.frame(CELLS=paste(proj_name, rownames(seurat_metadata), sep="_"), seurat_metadata)



Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec,  : 
  line 2 did not have 27 elements


/Users/nyquist/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in scan(file = file, what = what, sep = sep, quote = quote, dec = dec,  : 
  line 2 did not have 27 elements

  warnings.warn(x, RRuntimeWarning)


In [5]:
# TODO get the metadata object from R
mapping_options["cell level dataframe"] = pd.read_csv("/Users/nyquist/Dropbox (MIT)/Shalek Lab (Team folder conflict)/Projects/Gates/NIH.Vaccine.Route.Comparison.2019/R_combined_metadata.txt", sep="\t", index_col=0)

/Users/nyquist/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,20,21,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
mapping_options["cell level dataframe"]["VaccineRoute"].unique()

array(['IDLow', 'Naive', 'AE', 'IV', 'IDHigh'], dtype=object)

In [66]:
import numpy as np

In [71]:
mapping_options["cell level dataframe"]["celltype"] = ""
mapping_options["cell level dataframe"].loc[mapping_options["cell level dataframe"]["Week25CellType2"].isna(),"celltype"] = mapping_options["cell level dataframe"].loc[mapping_options["cell level dataframe"]["Week25CellType2"].isna(),"Week13CellType"]
mapping_options["cell level dataframe"].loc[mapping_options["cell level dataframe"]["Week13CellType"].isna(),"celltype"] = mapping_options["cell level dataframe"].loc[mapping_options["cell level dataframe"]["Week13CellType"].isna(),"Week25CellType2"]





## Scanpy Step 1: Dimensionality reduction 'cluster files'

The SCP expects one file per dimensionality reduction visualization, so the following code chunk will save a file for each dimensionality reduction stored in the ``obsm`` attribute of your ``AnnData`` object. The ``print(adata)`` command above should show you the available dimensionality reduction values. 


The upload interface allows you to input a minimum and maximum for each axis of each dimensionailty reduction. This next code chunk will also print out those results so you can enter them in the upload interface.

In [4]:
if len(anndata_path) > 0:
    conv.save_cluster_dfs(adata, output_dir)
    # TODO: save output file names


min for X_pca X is -12.995701
max for X_pca X is 56.850597
min for X_pca Y is -12.995701
max for X_pca Y is 56.850597
min for X_umap X is -6.8291025920439345
max for X_umap X is 9.507524406034713
min for X_umap Y is -6.8291025920439345
max for X_umap Y is 9.507524406034713


In [ ]:
# TODO: convert merged_metadata to dataframe



## Scanpy Step 2: expression matrices

Next you will save your expression matrix in the SCP format



In [6]:
if len(anndata_path) > 0:
    out_df = conv.make_expression_df(adata)
    out_df.to_csv(os.path.join(output_dir, "expression_file.txt.gz"), compression='gzip')

# Both Scanpy and Seurat: Set up metadata

Now we will convert metadata in your object to the correct metadata format and naming scheme for Alexandria


This includes 3 parts:

* Global Metadata

* Alexandria structured metadata to be mapped from your metadata tables

* Units for numeric metadata

* Unstructured metadata mapped from your metadata tables


In [73]:
# TODO: adjust this if you are not looking for cell level and are doing sample level instead
cell_level_metadata = pd.DataFrame(index=mapping_options["cell level dataframe"].index)

### Step 3a: Starting with global metadata

These metadata attributes must be the same for all of your columns

In [74]:
global_attributes = {}

### Global Metadata: Select species for this data

Type a search term in the drop down below (ex. 'homo' for human or 'mus' for mouse)

then run the next code section and select the value from that drop-down

If you do not see the value you were looking for and would like to search again, type in a new value in the text box and *rerun the code block that generates the species*

In [75]:
s =widgets.Text(
    value='Species',
    placeholder='Species search',
    description='Species:',
    disabled=False
)
display(s)
print("enter a search term for the species in the box above then run the next notebook cell to see a dropdown of search results")

Text(value='Species', description='Species:', placeholder='Species search')

enter a search term for the species in the box above then run the next notebook cell to see a dropdown of search results


In [76]:
list_for_dropdown, name_id_dict=uh.query_search_term('ncbitaxon',s.value)
m=uh.choose_metadata_name_dropdown(list_for_dropdown,"species")
display(m)

Dropdown(description='species', options=(('Theretra rhesus: ', 'NCBITaxon_1088310'), ('Polites rhesus: ', 'NCB…

### Once you are happy with your selection above, run the code block below!!

In [77]:
global_attributes['species']=m.value
global_attributes['species__ontology_label'] = name_id_dict[m.value]

### Global Metadata: Select library preparation protocol for this data

Just select from the dropdown below then run the code block under it

In [78]:
print("this takes a second if your internet connection isn't super fast")
list_for_dropdown, name_id_dict = uh.query_all_values_under_root("efo","EFO_0001457")
exp_method_dpdn=uh.choose_metadata_name_dropdown(list_for_dropdown,"experimental method")
display(exp_method_dpdn)
print("select from dropdown above then run the next notebook cell. If you want to change the selection, you need to re-run the cell below")

this takes a second if your internet connection isn't super fast
11


Dropdown(description='experimental method', options=(('RARseq: Restriction site associated RNA sequencing', 'E…

select from dropdown above then run the next notebook cell. If you want to change the selection, you need to re-run the cell below


In [79]:
global_attributes['library_preparation_protocol']=exp_method_dpdn.value
global_attributes['library_preparation_protocol__ontology_label'] = name_id_dict[exp_method_dpdn.value]

print(name_id_dict[exp_method_dpdn.value] +" saved as library preparation protocol")

Seq-Well saved as library preparation protocol


In [80]:
# explicitly saving the cellID column
for k,v in global_attributes.items():
    cell_level_metadata[k] = v

cell_level_metadata["CellID"] = cell_level_metadata.index

In [81]:
# reading in all the metadata in the convention
#TODO: make this work with JSON, for now it is just a table copy and pasted from the google sheets
metadata_info = pd.read_csv("metadata_name_type_info.tsv",sep="\t",index_col=0)
metadata_info["is unit"] = ["unit" in i for i in metadata_info.index] # removing units from this so people are less confused
# removing the "label" types because we will add those automatically
available_metadata = metadata_info[~metadata_info["class"].isin(["unit_label", "ontology_label"]) & ~metadata_info["is unit"]].index

# we already added species and library prep so drop those as well
available_metadata=available_metadata.drop("species")
available_metadata=available_metadata.drop('library_preparation_protocol')

## Step 3b: Renaming metadata from your dataframe and files

Now we will facilitate mapping any of your metadata to the Alexandria metadata convention.

If there is required (or optional) metadata that you would like to add that is not already in one of your files AND is an ontology or controlled list metadata type AND follows the same pattern as some other metadata in your files (ex. sample level or donor level), you can do so by mapping the metadata from some unrelated metadata.

For example, if you recorded the organ of each sample but did not save it as an explicit column, you can choose the sample column and map from that


You should run this group of cells once per metadata source (as in sample level file, your cell level dataframe, etc)

In [ ]:
# this is the workhorse cell of the metadata adder!
# TODO: make the labels for the dropdowns show the whole text
# TODO: add some indication that you should just go back to the top and re-run for multiple files
# TODO: give indication of which metadata has been added
# TODO: non-controlled list doesnt bring you back to the starting dropdown
meta_addr = ma.MetadataAdder(mapping_options, available_metadata, metadata_info, cell_level_metadata)

In [276]:
cell_level_metadata = meta_attr.cell_level_metadata

Now if you have any more files to add, go back to _Step 3b_. If not go to the next cell

## Step 3c: Check that all required keys are mapped

In [279]:
required_keys = metadata_info.loc[metadata_info["required"]=="Yes"].index

str

In [291]:
for k in required_keys:
    if k not in cell_level_metadata.columns:
        #default?
        default_val = metadata_info.loc[k,"default"]
        if type(default_val) is str: # this is sketchy but right now all the defaults are strings so whatevs
            cell_level_metadata[k] = default_val
        else:
            print("You need to add a value for "+k+" before proceeding!")

You need to add a value for is_living before proceeding!
You need to add a value for sample_type before proceeding!
You need to add a value for disease__ontology_label before proceeding!
You need to add a value for CellID before proceeding!


To add these required values you should go back to Step 3b. 

## Step 3d: All numeric metadata needs units, so now add units...

In [303]:
# make a list of unit metadata that you have
my_metadata_info = metadata_info.loc[cell_level_metadata.columns]
unit_dropdowns = {}
name_id_dicts = {}
for dep_metadata in my_metadata_info.loc[~my_metadata_info["dependent"].isna()].index:
    m_name = metadata_info.loc[dep_metadata,"dependent"]
    
    if metadata_info.loc[m_name, "class"] == "ontology":
        ont = metadata_info.loc[m_name, "ontology"].split("/")[-1]
        list_for_dropdown, name_id_dict = uh.query_all_values_under_root(ont,metadata_info.loc[m_name, "ontology_root"])
        unit_dropdowns[m_name] = uh.choose_metadata_name_dropdown(list_for_dropdown,m_name)
        name_id_dicts[m_name] = name_id_dict
    else:
        unit_dropdowns[m_name] = widgets.Text(
                    value="type unit here",
                    placeholder='type unit here',
                    description=m_name,
                    disabled=False
                )
    
for n,v in unit_dropdowns.items():
    display(v)

1


Dropdown(description='organism_age__unit', options=(("month: A time unit which is approximately equal to the l…

In [305]:
unit_values = {}
for n,v in unit_dropdowns.items():
    cells_with_dep = cell_level_metadata.loc[~cell_level_metadata[metadata_info.loc[n,"dependency"]].isna()].index
    val = v.value
    if n in name_id_dicts:
        cell_level_metadata.loc[cells_with_dep, n+"__ontology_label"] = name_id_dicts[n][val]
    cell_level_metadata.loc[cells_with_dep,n] = val



(13341, 17)

(4447, 8)

## Step 3e: Add in any unstructured metadata that you want to add (and their types)

In [ ]:
print("here are all the metadata columns in your cell level files:")

print(mapping_options["cell level dataframe"].columns)



In the dictionary below, replace the keys with column names in the choices printed above and replace the values with the type: group or numeric, of the column. Add as many as you would like

In [ ]:
unstructured_metadata_types = {"columnname 1":"numeric","columnname 2": "group"}

In [ ]:
for m in unstructured_metadata_types:
    cell_level_metadata[m] = mapping_options["cell level dataframe"].loc[cell_level_metadata.index, m]

## Map the structured Alexandria metadata types

In [45]:
metadata_info['alexandria type']=metadata_info['type'].map({"string":"group","number":"numeric","boolean":"group"})

types_row = pd.DataFrame(index= ["TYPE"], columns = cell_level_metadata.columns)

for column in cell_level_metadata.columns:
    if column in metadata_info.index:
        types_row.loc["TYPE",column] = metadata_info.loc[column,'alexandria type']
        
    else:
        types_row.loc["TYPE", column] = unstructured_metadata_types[column]


final_metadata_dataframe = pd.concatenate([types_row, cell_level_metadata])

final_metadata_dataframe.index.name = "CELL"


array(['string', 'number', 'boolean'], dtype=object)

# Write file to Alexandria format

In [ ]:
final_metadata_dataframe.to_csv(prefix+"structured_metadata.csv")

# Now what?

This is a description of how to figure out how to upload these files and a like to SCP (and how to get it in the alexandria namespace)